# Aggregate features for training in a DataFrame

In [1]:
import pandas as pd
import numpy as np
import os.path
import json

In [8]:
# Extract feature data and filter the existing data. Playout data from Rebox is stored in 
#"kristine_playouts_filtered.csv" file. Extracted features from the audio files are stored in idividual JSON files.
# This script extracts and stores feature data in data frame. It also stores metadata from Rebox in a data frame 
# that matches the feature data frame in the way data is ordered.
count=0
data = pd.read_csv("kristine_playouts_filtered.csv",
                   names=['Name','Station', 'ID', 'Show','Date','Number', 'Song', 'Artist', 'Track_ID'])
df = (pd.DataFrame(data))
df_meta = pd.DataFrame()
df_features = pd.DataFrame()
sample_features = {}
import time
for i in range(len(df.index)):
    startTime = time.clock()
    print (i)
    track_id = df['Track_ID'][i]
    extension = os.path.splitext(track_id)[0]
#     print(extension)
    file_path = "data/"+extension+"_ll.json"
# Check if feature file exists for a particular song. If yes store it in data frame row.
    if os.path.exists(file_path):
#             print(file_path)
            count=count+1
#            print(count)
#         Extract features from the Json file
            data = json.load(open(file_path))
            df_meta = df_meta.append(df.iloc[i])
            sample_json = data['lowlevel']
            for key in sample_json:
                if not(isinstance(sample_json[key], dict)):
                    sample_features["%s" % (key)] = sample_json[key]
                else:    
                    for key2 in sample_json[key]:
                        k=0
                        if not(isinstance(sample_json[key][key2], list)):
                            sample_features["%s_%s" % (key, key2)] = sample_json[key][key2]
                        elif isinstance(sample_json[key][key2], list):
                            for num in sample_json[key][key2]:
                                e=0
                                if isinstance(num, list):
                                    for num_num in num:
                                        e=e+1
                                        sample_features["%s_%s_%d_%d" % (key, key2,k,e)] = num_num
                                else:
                                    sample_features["%s_%s_%d" % (key, key2,k)] = num
                                    k=k+1
            df_feat = pd.DataFrame(sample_features, index=[count])  
            df_features=df_features.append(df_feat)
    endTime = time.clock()
    print("time: %s" % (endTime - startTime))
print('Done')
   

0
time: 0.010358
1
time: 0.000133
2


TypeError: list indices must be integers, not unicode

In [7]:
df_feat

,spectral_complexity_min,spectral_complexity_max,spectral_complexity_dvar2,spectral_complexity_median,spectral_complexity_dmean2,spectral_complexity_dmean,spectral_complexity_var,spectral_complexity_dvar,spectral_complexity_mean,silence_rate_20dB_min,...,hfc_mean,barkbands_crest_min,barkbands_crest_max,barkbands_crest_dvar2,barkbands_crest_median,barkbands_crest_dmean2,barkbands_crest_dmean,barkbands_crest_var,barkbands_crest_dvar,barkbands_crest_mean
30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Store feature and metadata data frames to csv files
df_features.to_csv('df_features.csv')
df_meta.to_csv('df_meta.csv')